# Generating Image Overlays of Wx Variables

First obtain the maximum and minimum values of Latitude and Longitude so that the image can be placed in Leaflet.

In [13]:
lat_min = float(lats.min())
lat_max = float(lats.max())
lon_min = float(lons.min())
lon_max = float(lons.max())

print(f"Image bounds: SW=({lat_min}, {lon_min}), NE=({lat_max}, {lon_max})")


Image bounds: SW=(-3.236724853515625, 44.73521041870117), NE=(43.468475341796875, 111.26478576660156)


Lets generate the images.

## Surface Winds

In [14]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from matplotlib.colors import LinearSegmentedColormap
from scipy.ndimage import gaussian_filter
from tqdm import notebook


# Load WRF data and Calculate Wind Speed
ds = xr.open_dataset("/Users/manaruchi/Desktop/WeatherDataViz/raw_data/AFCNWP_WRF_model_output_00UTC.nc")
u = ds["U10"]
v = ds["V10"]
wind_speed = np.sqrt(u**2 + v**2) * 1.94384 # Wind Speed in Knots


# Generate hourly images for 3 days
tlimit = 72

for t in notebook.tqdm(range(tlimit), desc="Generating Hourly Surface Wind Speed Images.."):

    ws = wind_speed.isel(Time=t).values
    lats = ds["XLAT"].isel(Time=t).values
    lons = ds["XLONG"].isel(Time=t).values
    
    # Smooth wind speed with Gaussian filter (sigma controls smoothness)
    ws_smooth = gaussian_filter(ws, sigma=1)
    
    # Windy color scale definition
    windy_colors = [
        "#0077FF",  # Blue
        "#00FFFD",  # Cyan
        "#4EFF3C",  # Green
        "#FFFF00",  # Yellow
        "#FFB600",  # Orange
        "#FF3E00",  # Red
        "#990000",  # Dark Red
    ]
    
    windy_cmap = LinearSegmentedColormap.from_list("windy", windy_colors)
    
    # Plot
    fig = plt.figure(figsize=(20, 16))
    ax = plt.axes(projection=ccrs.PlateCarree())
    
    ax.axis("off")
    ax.set_frame_on(False)
    fig.patch.set_alpha(0.0)
    
    # Compute image extent for imshow: [left, right, bottom, top]
    extent = [lons.min(), lons.max(), lats.min(), lats.max()]
    
    # Use imshow with interpolation for smooth result
    img = ax.imshow(
        ws_smooth,
        origin='lower',
        extent=extent,
        cmap=windy_cmap,
        vmin=0,
        vmax=50,
        interpolation='bilinear',   # 'bilinear' or 'bicubic' for smoothness
        alpha=0.8
    )
    
    ax.set_extent(extent, crs=ccrs.PlateCarree())
    
    plt.savefig(
        f"wind_speed_{t}_0.png",
        dpi=300,
        bbox_inches='tight',
        pad_inches=0,
        transparent=True
    )
    plt.close()

Generating Hourly Surface Wind Speed Images..:   0%|          | 0/72 [00:00<?, ?it/s]

## Generate Upper Air Winds

In [13]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from matplotlib.colors import LinearSegmentedColormap
from scipy.ndimage import gaussian_filter
from tqdm import notebook

# Load WRF NetCDF file
ds = xr.open_dataset("/Users/manaruchi/Desktop/WeatherDataViz/raw_data/AFCNWP_WRF_model_output_00UTC.nc")
print("Dataset Loaded...")
tlimit = 72 # hourly data for 3 days (24x3)

levels = [0,6,11,14,16,21,24,28,31] # Levels in concurrence with bottom_top values


for i,level in enumerate(levels):
    for t in notebook.tqdm(range(tlimit), desc=f"Generating Hourly Wind JSON. Level: {i+1}"):
        u = ds["U"].isel(Time=t, bottom_top=level)
        v = ds["V"].isel(Time=t, bottom_top=level)
        lats = ds["XLAT"].isel(Time=t).values
        lons = ds["XLONG"].isel(Time=t).values
    
        # Unstagger the Wind Data
        u_unstaggered = 0.5 * (u[:, :-1] + u[:, 1:])   # average in x-direction
        v_unstaggered = 0.5 * (v[:-1, :] + v[1:, :])   # average in y-direction

        u_arr = u_unstaggered.values * u_unstaggered.values
        v_arr = v_unstaggered.values * v_unstaggered.values
        final_arr = u_arr + v_arr
        final_arr = final_arr ** 0.5
        ws = final_arr * 1.94384
        # Smooth wind speed with Gaussian filter (sigma controls smoothness)
        ws_smooth = gaussian_filter(ws, sigma=1)
        
        # Windy color scale definition
        windy_colors = [
            "#0077FF",  # Blue
            "#00FFFD",  # Cyan
            "#4EFF3C",  # Green
            "#FFFF00",  # Yellow
            "#FFB600",  # Orange
            "#FF3E00",  # Red
            "#990000",  # Dark Red
            "#800080",  # Purple
        ]
        
        windy_cmap = LinearSegmentedColormap.from_list("windy", windy_colors)
        
        # Plot
        fig = plt.figure(figsize=(20, 16))
        ax = plt.axes(projection=ccrs.PlateCarree())
        
        ax.axis("off")
        ax.set_frame_on(False)
        fig.patch.set_alpha(0.0)
        
        # Compute image extent for imshow: [left, right, bottom, top]
        extent = [lons.min(), lons.max(), lats.min(), lats.max()]
        
        # Use imshow with interpolation for smooth result
        img = ax.imshow(
            ws_smooth,
            origin='lower',
            extent=extent,
            cmap=windy_cmap,
            vmin=0,
            vmax=50,
            interpolation='bilinear',   # 'bilinear' or 'bicubic' for smoothness
            alpha=0.8
        )
        
        ax.set_extent(extent, crs=ccrs.PlateCarree())
        
        plt.savefig(
            f"wind_speed_{t}_{i+1}.png",
            dpi=300,
            bbox_inches='tight',
            pad_inches=0,
            transparent=True
        )
        plt.close()

Dataset Loaded...


Generating Hourly Wind JSON. Level: 1:   0%|          | 0/72 [00:00<?, ?it/s]

Generating Hourly Wind JSON. Level: 2:   0%|          | 0/72 [00:00<?, ?it/s]

Generating Hourly Wind JSON. Level: 3:   0%|          | 0/72 [00:00<?, ?it/s]

Generating Hourly Wind JSON. Level: 4:   0%|          | 0/72 [00:00<?, ?it/s]

Generating Hourly Wind JSON. Level: 5:   0%|          | 0/72 [00:00<?, ?it/s]

Generating Hourly Wind JSON. Level: 6:   0%|          | 0/72 [00:00<?, ?it/s]

Generating Hourly Wind JSON. Level: 7:   0%|          | 0/72 [00:00<?, ?it/s]

Generating Hourly Wind JSON. Level: 8:   0%|          | 0/72 [00:00<?, ?it/s]

Generating Hourly Wind JSON. Level: 9:   0%|          | 0/72 [00:00<?, ?it/s]